In [1]:
import os


In [2]:
#finding current directory
%pwd

'c:\\Projects\\datascienceproject\\research'

In [4]:
#to change directory to parent one
os.chdir("../")
%pwd

'c:\\Projects\\datascienceproject'

In [5]:

# @dataclass: This decorator automatically creates an __init__ method, making it easier to initialize instances.
# The class defines configuration settings for data ingestion:
from dataclasses import dataclass
from pathlib import Path



#normal class we use self keyword , in data class below we do not use the self, here focus is on assigning value to variable, ex root_dir:path
#the below config will taken on config.yaml
@dataclass
class DataIngestionConfig:
    root_dir: Path          # root_dir: Directory where all data-related files are stored.
    source_URL: Path        # source_URL: URL to download the data from.
    local_data_file: Path   # local_data_file: Path where the downloaded file will be stored.
    unzip_dir: Path         # unzip_dir: Directory where the extracted files will be placed.


#Example usage
# config = DataIngestionConfig(
#     root_dir=Path("/data"),
#     source_URL=Path("https://example.com/data.zip"),
#     local_data_file=Path("/data/data.zip"),
#     unzip_dir=Path("/data/unzipped")
# )


In [15]:
from src.datascience.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.datascience.utils.common import read_yaml, create_directories
#configuration manager will load the input from src/datascience/constant
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):  #CONFIG_FILE_PATH WILL BE present in src\datascience\constants
        self.config=read_yaml(config_filepath)  #we want to read the yaml file so we will use read_yaml function which is defined in utils
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])  #artifacts_root is in  config.yaml, here we are creating artifact


    def get_data_ingestion_config(self)-> DataIngestionConfig:  #here we are returning dataingestion config which is above the path for root dir, etc
        config=self.config.data_ingestion  #data_ingestion is the key config.yaml file
        create_directories([config.root_dir]) #the first folder needed to create in config.yaml is root_directories

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config














In [16]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [17]:
##Component-Data ingestion component

#after getting the configuration details in data ingestion config, we should be downloading the entire zip file


# This class is responsible for downloading and extracting data.
# It takes a DataIngestionConfig object as input.



class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    #downloading the zip fil
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):   #request.urlretrieve(): Downloads the file from source_URL and saves it to local_data_file.
            filename, headers=request.urlretrieve(            #filename: Path where the file is saved. #headers: Response headers (metadata)
                url=self.config.source_URL,                   
                filename= self.config.local_data_file         
            )
            logger.info(f"{filename}download! with following info: \n{headers}")

        else:
            logger.info(f"Already exist")





    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extract the zip file to data directory
        function return none
        
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        















In [18]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-02-06 15:15:35,599: INFO: common: yaml file: config\config.yaml loaded succefully]
[2025-02-06 15:15:35,600: INFO: common: yaml file: params.yaml loaded succefully]
[2025-02-06 15:15:35,604: INFO: common: yaml file: schema.yaml loaded succefully]
[2025-02-06 15:15:35,605: INFO: common: created directory at artifacts]
[2025-02-06 15:15:35,606: INFO: common: created directory at artifacts/data_ingestion]
[2025-02-06 15:15:36,376: INFO: 478841809: artifacts/data_ingestion/data.zipdownload! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C8AE:EF5A0:349E71:3A10F3:67A51864
Accept-Ranges: bytes
Date: Thu, 06 Feb 2025 2